In [ ]:
#%%

import quantstats        as qs
import yfinance          as yf
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt

pd.options.mode.copy_on_write = True

In [ ]:
#%%

df_ = yf.download("SPY", start="1992-01-01", interval="1d", actions=True)

df_['Open'     ] = df_['Open'     ].round(4)
df_['High'     ] = df_['High'     ].round(4)
df_['Low'      ] = df_['Low'      ].round(4)
df_['Close'    ] = df_['Close'    ].round(4)
df_['Adj Close'] = df_['Adj Close'].round(4)

df_

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,,
1993-01-29,43.9688,43.9688,43.7500,43.9375,24.7637,1003200,0.0,0.0,0.0
1993-02-01,43.9688,44.2500,43.9688,44.2500,24.9399,480500,0.0,0.0,0.0
1993-02-02,44.2188,44.3750,44.1250,44.3438,24.9927,201300,0.0,0.0,0.0
1993-02-03,44.4062,44.8438,44.3750,44.8125,25.2569,529400,0.0,0.0,0.0
1993-02-04,44.9688,45.0938,44.4688,45.0000,25.3626,531500,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2024-04-04,523.5200,523.8700,512.7600,513.0700,513.0700,96858100,0.0,0.0,0.0
2024-04-05,514.4600,520.4400,514.0100,518.4300,518.4300,74482100,0.0,0.0,0.0
2024-04-08,519.1500,520.1800,517.8900,518.7200,518.7200,48401800,0.0,0.0,0.0


In [ ]:
#%%

df_.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7855 entries, 1993-01-29 to 2024-04-10
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Open           7855 non-null   float64
 1   High           7855 non-null   float64
 2   Low            7855 non-null   float64
 3   Close          7855 non-null   float64
 4   Adj Close      7855 non-null   float64
 5   Volume         7855 non-null   int64  
 6   Dividends      7855 non-null   float64
 7   Stock Splits   7855 non-null   float64
 8   Capital Gains  7855 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 613.7 KB


In [ ]:
#%%

df_[df_['Dividends']>0]

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,,
1993-03-19,45.2812,45.2812,45.0312,45.0312,25.5001,66900,0.213,0.0,0.0
1993-06-18,44.8438,44.8438,44.5000,44.5000,25.3778,58500,0.318,0.0,0.0
1993-09-17,45.8750,45.9062,45.7500,45.8125,26.2891,200900,0.286,0.0,0.0
1993-12-17,46.4062,46.5938,46.3750,46.5625,26.9024,104700,0.317,0.0,0.0
1994-03-18,46.7500,47.0312,46.7188,46.9688,27.2937,365500,0.271,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2023-03-17,393.2200,394.4000,388.5500,389.9900,384.4231,140553400,1.506,0.0,0.0
2023-06-16,443.0200,443.6100,438.9700,439.4600,434.7961,114121300,1.638,0.0,0.0
2023-09-15,447.1400,447.4800,442.9200,443.3700,440.2119,111761400,1.583,0.0,0.0


In [ ]:
#%%

df_[df_['Stock Splits']>0]

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,,


In [ ]:
#%%

df_['FillPrice'] = df_['Open' ].shift(-1)
df_['Date'     ] = df_.index
df_['DateIn'   ] = df_['Date' ].shift(-1)
df_['DateOut'  ] = df_['Date' ].shift(-1)

df_.fillna(method='ffill', inplace=True)

<ipython-input-6-720fff6565e0>:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_.fillna(method='ffill', inplace=True)


In [ ]:
#%%

# Strategy backtester

def backtest(df_, test_number, ma_fast, ma_slow):
    print(f"Testing {test_number}, MA_FAST : {ma_fast}, MA_SLOW : {ma_slow}")
    df = df_.copy()

    df['EntrySetup'] = 0
    df['ExitRule'  ] = 0

    df['MaFast'] = df['Close'].rolling(ma_fast).mean()
    df['MaSlow'] = df['Close'].rolling(ma_slow).mean()

    df.dropna(inplace=True)

    df.loc[((df['MaFast']>df['MaSlow']) & (df['MaFast'].shift(1)<=df['MaSlow'].shift(1))), 'EntrySetup'] = 1
    df.loc[((df['MaFast']<df['MaSlow']) & (df['MaFast'].shift(1)>=df['MaSlow'].shift(1))), 'ExitRule'  ] = 1

    df.loc[df.index[ 0], 'EntrySetup'] = 1
    df.loc[df.index[-1], 'ExitRule'  ] = 1



    # Position tracking
    date_in          = None
    entry_fill_price = 0
    date_out         = None
    exit_fill_price  = 0
    position_history = []

    for index, row in df.iterrows():
        # EntrySetup
        if row['EntrySetup'] == 1:
            date_in          = row['DateIn'   ]
            entry_fill_price = row['FillPrice']
        # ExitRuel
        if row['ExitRule'  ] == 1:
            date_out        = row['DateOut'  ]
            exit_fill_price = row['FillPrice']
            pct_change      = (exit_fill_price - entry_fill_price)/entry_fill_price
            bars            = len(df[date_in:date_out])-1
            position_history.append((
                date_in, 
                date_out, 
                entry_fill_price, 
                exit_fill_price, 
                pct_change,
                bars
                ))

    position_df = pd.DataFrame(position_history, columns=['DateIn', 'DateOut', 'PriceIn', 'PriceOut', 'Return', 'Bars'])
    position_df = position_df.set_index(pd.DatetimeIndex(position_df['DateIn']))


    initial_capital = 10000.0

    df['Return'] = df['Close'].pct_change()

    df['StratReturn'] = 0.0
    for index, row in position_df.iterrows():
        sub_df = df[row['DateIn']:row['DateOut']]["Return"]
        df.loc[sub_df.index, 'StratReturn'] = sub_df

    df['GrowthFactor'] = (1 + df['StratReturn']).cumprod()
    df['DollarEquity'] = initial_capital*df['GrowthFactor']


    return {
        'final_capital'    : round(df.iloc[-1]['DollarEquity'], 2),
        'sharpe_ratio'     : round(qs.stats.sharpe(df['StratReturn']), 4),
        'maximum_drawdown' : round(qs.stats.max_drawdown(df['StratReturn']) * -100.0, 2),
        'win_rate'         : round(qs.stats.win_rate(df['StratReturn']), 4),
        'sortino_ratio'    : round(qs.stats.sortino(df['StratReturn']), 4),
        'cagr'             : round(qs.stats.cagr(df['StratReturn']), 4),
        'profit_factor'    : round(qs.stats.profit_factor(df['StratReturn']), 4),
        'prob_sr'          : round(qs.stats.probabilistic_sharpe_ratio(df['StratReturn']), 4),
    }

In [ ]:
#%%

# Parameter Optimization

counter       = 0
backtest_list = []

for ma_fast in range(10, 90, 10):
    for ma_slow in range(100, 350, 50):
        counter += 1
        result   = backtest(df_=df_, test_number=counter, ma_fast=ma_fast, ma_slow=ma_slow)
        backtest_list.append((
            counter,
            result['final_capital'   ],
            result['sharpe_ratio'    ],
            result['maximum_drawdown'],
            result['win_rate'        ],
            result['sortino_ratio'   ],
            result['cagr'            ], 
            result['profit_factor'   ],
            result['prob_sr'         ],
            ma_fast,
            ma_slow
        ))

backtests_df = pd.DataFrame(backtest_list,
                            columns=[
                                'test_number',
                                'Final Capital',
                                'Sharpe Ratio',
                                'Maximum Drawdown',
                                'Win Rate',
                                'Sortino Ratio',
                                'Compounded Annual Growth Rate',
                                'Profit Factor',
                                'Probabilistic Sharpe Ratio',
                                'Fast MA',
                                'Slow MA'
                            ])

backtests_df.set_index('test_number', inplace=True)

Testing 1, MA_FAST : 10, MA_SLOW : 100
Testing 2, MA_FAST : 10, MA_SLOW : 150
Testing 3, MA_FAST : 10, MA_SLOW : 200
Testing 4, MA_FAST : 10, MA_SLOW : 250
Testing 5, MA_FAST : 10, MA_SLOW : 300
Testing 6, MA_FAST : 20, MA_SLOW : 100
Testing 7, MA_FAST : 20, MA_SLOW : 150
Testing 8, MA_FAST : 20, MA_SLOW : 200
Testing 9, MA_FAST : 20, MA_SLOW : 250
Testing 10, MA_FAST : 20, MA_SLOW : 300
Testing 11, MA_FAST : 30, MA_SLOW : 100
Testing 12, MA_FAST : 30, MA_SLOW : 150
Testing 13, MA_FAST : 30, MA_SLOW : 200
Testing 14, MA_FAST : 30, MA_SLOW : 250
Testing 15, MA_FAST : 30, MA_SLOW : 300
Testing 16, MA_FAST : 40, MA_SLOW : 100
Testing 17, MA_FAST : 40, MA_SLOW : 150
Testing 18, MA_FAST : 40, MA_SLOW : 200
Testing 19, MA_FAST : 40, MA_SLOW : 250
Testing 20, MA_FAST : 40, MA_SLOW : 300
Testing 21, MA_FAST : 50, MA_SLOW : 100
Testing 22, MA_FAST : 50, MA_SLOW : 150
Testing 23, MA_FAST : 50, MA_SLOW : 200
Testing 24, MA_FAST : 50, MA_SLOW : 250
Testing 25, MA_FAST : 50, MA_SLOW : 300
Testing 2

In [ ]:
#%%

backtests_df

,Final Capital,Sharpe Ratio,Maximum Drawdown,Win Rate,Sortino Ratio,Compounded Annual Growth Rate,Profit Factor,Probabilistic Sharpe Ratio,Fast MA,Slow MA
test_number,,,,,,,,,,
1,68896.37,0.5859,35.73,0.5433,0.8181,0.0442,1.1294,0.9994,10,100
2,67834.91,0.5775,28.18,0.5424,0.8033,0.0441,1.1264,0.9992,10,150
3,93869.35,0.6563,19.03,0.5451,0.9110,0.0521,1.1450,0.9998,10,200
4,86939.47,0.6273,20.49,0.5448,0.8684,0.0506,1.1371,0.9997,10,250
5,85041.54,0.6070,27.70,0.5433,0.8379,0.0504,1.1325,0.9995,10,300
6,47251.30,0.4730,40.02,0.5403,0.6520,0.0354,1.1040,0.9953,20,100
7,67222.23,0.5586,30.01,0.5432,0.7740,0.0439,1.1240,0.9989,20,150
8,98473.05,0.6445,30.11,0.5474,0.8865,0.0533,1.1462,0.9997,20,200
9,86499.41,0.6044,30.63,0.5449,0.8345,0.0505,1.1357,0.9994,20,250
